## Natural Gas Deliverability Problem

In [ ]:
#The following information about a newly discovered sweet, dry gas pool in a Devonian reef is provided

### Parameters

In [ ]:
# planimetered rock volume = 93,064 acre-feet (area = 6,400 acres) 
# weighted average pool porosity, φ = 23% 
# weighted average water saturation, Sw = 20% 
# average pool depth = 6,000 feet 
# initial reservoir pressure, pi = 2,600 psia 
# no gas analysis, but gas density, Yg = 0.65 

In [ ]:
#The key difference between sweet and sour natural gas is that the sweet natural gas contains trace amounts of hydrogen sulfide,
#whereas the sour natural gas contains large amounts of hydrogen sulfide. Moreover, sweet natural gas is non-corrosive, 
#less acidic, and requires little refining.

### 1. Calculate original raw gas-in-place and original recoverable raw gas-in-place.

In [ ]:
#Use Brown(1984) or Joshi(1991) correlation??

def pseudocritical_parameters(gas_density):
    Ppc = 708.75 - 57.5 * gas_density
    Tpc = 169 + 314 * gas_density
    return Ppc, Tpc

In [ ]:
pseudocritical_parameters(0.65)

In [ ]:
#Use Abandonment Pressure rule of thumb!

#Pa = 66 psia + 66 psia of depth (Field) 
#1,000 feet 
#or 
#Pa = 240 kPaa + 80 kPaa of depth (SI) 
#100 m

In [ ]:
def abandonment_pressure(average_depth):
    Pa = 66 + 66/1000 * average_depth
    return Pa

In [ ]:
print(round(abandonment_pressure(6000),2), "Psia")

In [ ]:
#Calculate pseudoreduced pressures and temperature
#Use temperature gradients for reference depth(Tf = 150 °F, 610 °R)

def pseudoreduced_initial_pressure(initial_res_pres, pseudocrit_pres):
    Ppri = initial_res_pres/pseudocrit_pres
    return Ppri

def pseudoreduced_abandon_pressure(abandon_pres, pseudocrit_pres):
    Ppra = abandon_pres/pseudocrit_pres
    return Ppra
print(round(pseudoreduced_initial_pressure(2600, 670),2), round(pseudoreduced_abandon_pressure(245, 670),2))

In [ ]:
#Use link https://petrolcalc.blogspot.com/2015/07/standing-and-katz-z-chart-reader.html

In [ ]:
def pseudoreduced_temperature(temp_grad, pseudocrit_temp):
    Tpr = temp_grad/pseudocrit_temp
    return Tpr

round(pseudoreduced_temperature(610, 373),2)

"https://petrolcalc.blogspot.com/2015/07/standing-and-katz-z-chart-reader.html"

In [ ]:
#Use link https://petrolcalc.blogspot.com/2015/07/standing-and-katz-z-chart-reader.html 
#to determine compressibility factor(zi and za)
#zi = 0.83 and za = 0.97

In [ ]:
#Original Raw Gas-in-Place (ORGIP) = 
#43,560* × A × h × Ø × (1−Swi) _________________________, standard cubic feet (scf) (3.3) Bgi
#where A = drainage area, acres (1 acre = 43,560 square feet) 
#h = net pay thickness, feet 
#[note: A × h = Vb, Rock (or Bulk) Volume, acre-feet] 
#Ø = porosity, fraction 
#Swi = connate water saturation at initial conditions, fraction 
#Bgi = natural gas formation volume factor (FVF) at initial conditions, ft3/scf 
#*Use 43.56 for units of Mscf or 0.04356 for units of MMscf.

In [ ]:
# Calculate Natural Gas Formation Volume Factor (Bg) 

#1/Bg = (520/14.65 * Tf)*p/z 

def ogas_FVF (temp_grad, init_press, comp_fac):
    GFVF = 520/(14.65 * temp_grad) * init_press/comp_fac
    return GFVF

round(ogas_FVF(610, 2600, 0.83),2)

In [ ]:
def Original_Raw_Gas_In_Place (rock_volume, porosity, water_saturation):
    ORGIP = 0.00004356 * rock_volume * porosity * (1-water_saturation) * 182.28
    return ORGIP
print(round(Original_Raw_Gas_In_Place (93064, 0.23, 0.2),2), "Bcf")

In [ ]:
def rgas_FVF (temp_grad, init_press, comp_fac, aband_press, aband_comp):
    GFVF = 520/(14.65 * temp_grad) * (init_press/comp_fac - aband_press/aband_comp)
    return GFVF

round(rgas_FVF(610, 2600, 0.83, 462, 0.97),2)

In [ ]:
def Recoverable_Gas_In_Place (rock_volume, porosity, water_saturation):
    ORGIP = 0.00004356 * rock_volume * porosity * (1-water_saturation) * 154.56
    return ORGIP
print(round(Recoverable_Gas_In_Place (93064, 0.23, 0.2),2), "Bcf")

### 2. P/Z VS Pressure (Bottom Hole Pressure)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
from matplotlib.pyplot import figure

# initialize data for p/z vs. pressure plot.
pz_table = {'Pressure':[2600, 462], 'Ppr':[3.88, 0.37], 'Tpr':[1.64, 1.64],'z':[0.83, 0.97], 'p/z(Psia)':[3132, 253] }
df = pd.DataFrame(pz_table, index =['Initial Reservoir Pressure','Abandonment Pressure'])
df

In [ ]:
plt.figure(figsize=(12, 10))
plt.plot(df['Pressure'], df['p/z(Psia)'], linestyle='--', color='green', marker='o')
plt.title('P/Z Vs Pressure')
plt.xlabel('Bottom Hole Pressure(Psia)')
plt.ylabel('p/z(Psia)')
plt.xlim([0, 3000])
plt.ylim([0, 3500])
#plt.grid()
plt.show()

### 3. P/Z VS Cumulative Gas Production (Bcf)

In [ ]:
cum_table = {'prd_cum':[0, 115, 135.96], 'p/z(Psia)':[3132, 462, 0] }
df2 = pd.DataFrame(cum_table)
df2

In [ ]:
plt.figure(figsize=(12, 10))
plt.plot(df2['prd_cum'], df2['p/z(Psia)'], linestyle='--', color='red', marker='o')
plt.title('P/Z Vs Cumulative Gas Production')
plt.xlabel('Cumulative Gas Production(Bcf)')
plt.ylabel('p/z(Psia)')
plt.annotate("Recoverable Gas Reserves = 115 Bcf", (120, 460),  xytext=(150, 500),arrowprops=dict(facecolor='black', shrink=0.01))
plt.xlim([0, 140])
plt.ylim([0, 3500])
plt.grid()
plt.show()

In [ ]:
#Calculate a shut-in BHP versus WHP plot ranging from original reservoir pressure 
#to abandonment pressure using Cullender and Smith Method
# pi = 2,600 psi zi = 0.83 Tf = 610 °R

In [ ]:
def coef (gas_density, pool_depth):
    x = 0.01875 * gas_density * pool_depth 
    return x
coef(0.65, 6000)

In [ ]:
#calculate I parameter
def param (temp, comp, press_ini):
    I = temp * comp / press_ini 
    return I
round(param(610, 0.83, 2600),3)

In [ ]:
#Assume Imp = Iws
def mid_point_pres (press_ini):
    Pmp = press_ini - (coef(0.65, 6000)/(2 * param(610, 0.83, 2600)))
    return Pmp
print(round(mid_point_pres (2600),2), "Psia")

In [ ]:
Ppr = mid_point_pres (2600)/671.374
round(Ppr,1)

In [ ]:
Tmp = (40+150)/2 + 460
Tpr = Tmp/373
round(Tpr,2)
Zmp = 0.765

In [ ]:
Imp = Tmp * Zmp / mid_point_pres (2600)
round(Imp,3)

In [ ]:
def mid_point_pres (press_ini):
    Pmp = press_ini - (coef(0.65, 6000)/(0.176 + param(610, 0.83, 2600)))
    return Pmp
print(round(mid_point_pres (2600),0), "Psia")
